In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV, Lasso, Ridge
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/캡스톤/2022/코드파일/0218_모델최적화/최종_누계비추가.csv")
df

,년,월,배추상품,배추중품,무상품,무중품,건고추상품,건고추중품,양파상품,양파중품,...,신선식품지수 전년누계비,평균기온(°C),평균최고기온(°C),평균최저기온(°C),평균상대습도(%),월합강수량(00~24h만)(mm),평균풍속(m/s),합계 일사량(MJ/m2),총지수 전년동월비,신선식품지수 전년동월비
0,2000,1,1,0,0,0,0,0,0,0,...,5.5,-2.1,1.7,-5.7,64,42.8,2.6,130.93,1.9,5.5
1,2000,1,1,0,0,0,0,0,0,0,...,5.5,-1.2,3.3,-5.2,70,27.5,2.4,158.92,1.9,5.5
2,2000,1,1,0,0,0,0,0,0,0,...,5.5,1.1,5.1,-2.7,57,9.2,2.3,157.46,1.9,5.5
3,2000,1,1,0,0,0,0,0,0,0,...,5.5,1.0,5.3,-2.6,67,33.7,2.3,176.49,1.9,5.5
4,2000,1,1,0,0,0,0,0,0,0,...,5.5,4.2,8.8,0.5,53,25.9,3.7,262.77,1.9,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10555,2021,12,0,0,0,0,0,0,0,1,...,6.2,0.6,5.1,-3.5,62,7.9,2.3,257.25,3.7,6.7
10556,2021,12,0,0,0,0,0,0,0,1,...,6.2,1.9,7.6,-3.0,70,4.4,1.5,241.28,3.7,6.7
10557,2021,12,0,0,0,0,0,0,0,1,...,6.2,3.1,8.9,-2.1,57,0.1,2.5,308.90,3.7,6.7
10558,2021,12,0,0,0,0,0,0,0,1,...,6.2,4.1,9.4,0.0,71,7.1,1.2,288.88,3.7,6.7


In [ ]:
def 종류별지역별(df, 종류, 지역):
  df1 = df[(df[종류]== 1) & (df[지역]== 1)]
  df1 = df1.loc[(df).any(1), (df1!=0).any(0)] # 모든 값이 0인 열 제거
  return df1

In [ ]:
def 종류별(df, 종류, 종류_1):
  df1 = df[(df[종류]== 1) | (df[종류_1]== 1)]
  df1 = df1.loc[(df).any(1), (df1!=0).any(0)] # 모든 값이 0인 열 제거
  return df1

In [ ]:
양파 = 종류별(df, '양파상품', '양파중품')

In [ ]:
양파.columns

Index(['년', '월', '양파상품', '양파중품', '서울', '부산', '대구', '광주', '대전', '면적', '생산량',
       '도매가격', '물가지수', '총지수 전년누계비', '신선식품지수 전년누계비', '평균기온(°C)', '평균최고기온(°C)',
       '평균최저기온(°C)', '평균상대습도(%)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)',
       '합계 일사량(MJ/m2)', '총지수 전년동월비', '신선식품지수 전년동월비'],
      dtype='object')

In [ ]:
def 양파f(df):
  target = '도매가격'
  X = df.drop(target, axis=1)
  drop_col = ['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
  X = X.drop(drop_col, axis=1)
  print(drop_col)
  y = df[target]
  return X, y

In [ ]:
seed = 42

In [ ]:
X, y = 양파f(양파)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']


In [ ]:
양파상_서울 = 종류별지역별(df, '양파상품', '서울')
양파상_부산 = 종류별지역별(df, '양파상품', '부산')
양파상_대구 = 종류별지역별(df, '양파상품', '대구')
양파상_광주 = 종류별지역별(df, '양파상품', '광주')
양파상_대전 = 종류별지역별(df, '양파상품', '대전')
양파중_서울 = 종류별지역별(df, '양파중품', '서울')
양파중_부산 = 종류별지역별(df, '양파중품', '부산')
양파중_대구 = 종류별지역별(df, '양파중품', '대구')
양파중_광주 = 종류별지역별(df, '양파중품', '광주')
양파중_대전 = 종류별지역별(df, '양파중품', '대전')

In [ ]:
양파 = [양파상_서울,양파상_부산, 양파상_대구,양파상_광주,양파상_대전, 양파중_서울,양파중_부산, 양파중_대구,양파중_광주,양파중_대전]

In [ ]:
def ridge_func(df):
  for i in range(len(df)):
    print("-----------------------{}----------------------------------".format(i))
    X, y = 양파f(df[i])
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
    ridge = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100])
    ridge.fit(X_train, y_train)
    alpha = ridge.alpha_
    print("Best alpha :", alpha)

    print("훈련 세트의 정확도   : {:.4f}".format(ridge.score(X_train,y_train)))
    print("테스트 세트의 정확도 : {:.4f}".format(ridge.score(X_test,y_test)))

    y_pred = ridge.predict(X_test)
    # print("RMSE     : {:.4f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
    print('RMSLE    : {:.4f}'.format(np.sqrt(mean_squared_log_error(y_test, y_pred))))
    print()

In [ ]:
ridge_func(양파)

-----------------------0----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 100.0
훈련 세트의 정확도   : 0.9286
테스트 세트의 정확도 : 0.9757
RMSLE    : 0.0685

-----------------------1----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 10.0
훈련 세트의 정확도   : 0.9510
테스트 세트의 정확도 : 0.9387
RMSLE    : 0.0889

-----------------------2----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 100.0
훈련 세트의 정확도   : 0.9207
테스트 세트의 정확도 : 0.9314
RMSLE    : 0.1114

-----------------------3----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 1.0
훈련 세트의 정확도   : 0.9146
테스트 세트의 정확도 : 0.8696
RMSLE    : 0.1148

-----------------------4----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 10.0
훈련 세트의 정확도   : 0.8957
테스트 세트의 정확도 : 0.9192
RMSLE   

In [ ]:
def minmax_ridge_func(df):
  for i in range(len(df)):
    print("-----------------------{}----------------------------------".format(i))
    X, y = 양파f(df[i])
    minmax_scaler = MinMaxScaler()
    X_minmax = minmax_scaler.fit_transform(X)
    X_minmax = pd.DataFrame(X_minmax)
    X_train, X_test, y_train, y_test = train_test_split(X_minmax, y, random_state=seed)
    ridge = RidgeCV(alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100])
    ridge.fit(X_train, y_train)
    alpha = ridge.alpha_
    print("Best alpha :", alpha)

    print("훈련 세트의 정확도   : {:.4f}".format(ridge.score(X_train,y_train)))
    print("테스트 세트의 정확도 : {:.4f}".format(ridge.score(X_test,y_test)))

    y_pred = ridge.predict(X_test)
    # print("RMSE     : {:.4f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
    print('RMSLE    : {:.4f}'.format(np.sqrt(mean_squared_log_error(y_test, y_pred))))
    print()

In [ ]:
minmax_ridge_func(양파)

-----------------------0----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 0.1
훈련 세트의 정확도   : 0.9292
테스트 세트의 정확도 : 0.9770
RMSLE    : 0.0691

-----------------------1----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 0.01
훈련 세트의 정확도   : 0.9510
테스트 세트의 정확도 : 0.9385
RMSLE    : 0.0892

-----------------------2----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 0.01
훈련 세트의 정확도   : 0.9217
테스트 세트의 정확도 : 0.9333
RMSLE    : 0.1080

-----------------------3----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 0.001
훈련 세트의 정확도   : 0.9146
테스트 세트의 정확도 : 0.8697
RMSLE    : 0.1147

-----------------------4----------------------------------
['총지수 전년동월비', '신선식품지수 전년동월비', '신선식품지수 전년누계비', '합계 일사량(MJ/m2)']
Best alpha : 0.1
훈련 세트의 정확도   : 0.8951
테스트 세트의 정확도 : 0.9201
RMSLE    :

## pkl

In [ ]:
import pickle
import joblib 

In [ ]:
양파 = pd.read_csv("/content/drive/MyDrive/캡스톤/2022/데이터/mysql/양파.csv")
양파.columns

Index(['년', '월', '양파상품', '양파중품', '서울', '부산', '대구', '광주', '대전', '면적', '생산량',
       '도매가격', '물가지수', '총지수전년누계비', '신선식품지수전년누계비', '평균기온', '평균최고기온', '평균최저기온',
       '평균상대습도', '월합강수량', '평균풍속', '합계일사량', '총지수전년동월비', '신선식품지수전년동월비', '양파상중'],
      dtype='object')

양파는 minmax + ridge이므로

In [ ]:
def 양파f(df):
  target = '도매가격'
  X = df.drop(target, axis=1)
  drop_col = ['총지수전년동월비', '신선식품지수전년동월비', '신선식품지수전년누계비', '합계일사량']
  X = X.drop(drop_col, axis=1)
  print(drop_col)
  y = df[target]
  return X, y

In [ ]:
X, y = 양파f(양파)
minmax_scaler = MinMaxScaler()
X_minmax = minmax_scaler.fit_transform(X)
X_minmax = pd.DataFrame(X_minmax)

# Train a Decision Tree Classifier
lasso = LassoCV(alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100])
lasso.fit(X, y)

filename = '/content/drive/MyDrive/캡스톤/2022/데이터/pkl/202205/onion_ridge.pkl'
pickle.dump(lasso, open(filename, 'wb'))

['총지수전년동월비', '신선식품지수전년동월비', '신선식품지수전년누계비', '합계일사량']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1846039.5632675728, tolerance: 13024.007588636367
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4547555.764013366, tolerance: 13024.007588636367
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4944928.912363952, tolerance: 13024.007588636367
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4999321.